# Skin Lesion Classification to Cancer Melanoma Detection

*Skin cancer is a major public health problem, with over 5,000,000 newly diagnosed cases in the United States every year. Melanoma is the deadliest form of skin cancer, responsible for an overwhelming majority of skin cancer deaths. In 2015, the global incidence of melanoma was estimated to be over 350,000 cases, with almost 60,000 deaths. Although the mortality is significant, when detected early, melanoma survival exceeds 95%.*

## Input Data:
The data were obtained from The International Skin Imaging Collaboration (ISIC) Repository. The ISIC Archive contains the largest publicly available collection of quality controlled dermoscopic images of skin lesions.

The input data are dermoscopic lesion images in JPEG format, and all lesion images are named using the scheme ISIC_.jpg, where is a 7-digit unique identifier.

## Objective:

Evaluate the CNN Model Resnet50 performance to make predictions of disease classification within dermoscopic images, and compare the results obtained with the results achieved by Andrey Sorokin  (see the paper "*Lesion Analysis and Diagnosis with Mask-RCNN*", link: https://arxiv.org/pdf/1807.05979.pdf)

## Response Data

The response data are sets of binary classifications for each of the 7 disease states, indicating the diagnosis of each input lesion image.

Response data are all encoded within a single CSV file (comma-separated value) file, with each classification response in a row. File columns must be:

    image: an input image identifier of the form ISIC_
    MEL: “Melanoma” diagnosis confidence
    NV: “Melanocytic nevus” diagnosis confidence
    BCC: “Basal cell carcinoma” diagnosis confidence
    AKIEC: “Actinic keratosis / Bowen’s disease (intraepithelial carcinoma)” diagnosis confidence
    BKL: “Benign keratosis (solar lentigo / seborrheic keratosis / lichen planus-like keratosis)” diagnosis confidence
    DF: “Dermatofibroma” diagnosis confidence
    VASC: “Vascular lesion” diagnosis confidence

![alt text](https://challenge2018.isic-archive.com/wp-content/uploads/2018/04/task3.png)

## Uploading data

The files of interest belong to task 3 of ISIC2018 Challenge, and they were uploaded to my Google Drive account. They're available to download on ISIC2018 web too: (https://challenge2018.isic-archive.com/task3/)

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz

In [3]:
!pip install scikit-image
!pip install keras

In [4]:
!ls

adc.json  sample_data


In [0]:
##TASK 3_Uploading Images:

file_id = '1LSrdLKlX7xuFpfMt6jagmQR17iBGSijy'     
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("ISIC2018_Task3_Training_Input.zip")   ## 10015 imagenes .jpg

file_id = '1lb0ELD1v3Aww68hUwFN1sYch9TchOhWt'              
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("ISIC2018_Task3_Test_Input.zip")    ## 1512 imagenes .jpg

file_id = '1ZYe1nE1YK0ukq3fCVqQwDRGSvg5MNXJJ'            
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("ISIC2018_Task3_Validation_Input.zip")   ## 193 imagenes .jpg

##GroundTruth_Uploading:
file_id = '1VaXSolajW_6flYdvy6OF7Bl8yG0_j-uG'                
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("ISIC2018_Task3_Training_GroundTruth.zip")  #Aqui dentro de este .zip esta el GroundTruth.csv

file_id = '1PFThiByKuU0dY0Rj1bsqEPesIRngjYBT'          
downloaded = drive.CreateFile({'id': file_id})
downloaded.GetContentFile("ISIC2018_Task3_Training_LesionGroupings.csv") 

In [6]:
!ls

adc.json
ISIC2018_Task3_Test_Input.zip
ISIC2018_Task3_Training_GroundTruth.zip
ISIC2018_Task3_Training_Input.zip
ISIC2018_Task3_Training_LesionGroupings.csv
ISIC2018_Task3_Validation_Input.zip
sample_data


## Data preparation:

To place the unzipped ISIC 2018 data, I created a folder "datasets/ISIC2018/data", including the following subfolders:

    ISIC2018_Task3_Training_GroundTruth
         Include ISIC2018_Task3_Training_LesionGroupings.csv file. 
    ISIC2018_Task3_Training_Input
    ISIC2018_Task3_Validation_Input
    ISIC2018_Task3_Test_Input


In [0]:
mkdir datasets

In [0]:
!cd '/content/datasets/' && mkdir ISIC2018

In [9]:
!find '/content/datasets'

/content/datasets
/content/datasets/ISIC2018


In [0]:
import os, shutil
base_dir = "/content/datasets/ISIC2018"
data_dir = os.path.join(base_dir, "data")
small_data_dir = os.path.join(base_dir, "small")

In [0]:
os.mkdir(data_dir)
os.mkdir(small_data_dir)

In [12]:
pwd

'/content'

In [13]:
!find '/content'

/content
/content/.config
/content/.config/logs
/content/.config/logs/2018.11.20
/content/.config/logs/2018.11.20/18.05.57.262344.log
/content/.config/logs/2018.11.20/18.06.10.838966.log
/content/.config/logs/2018.11.20/18.06.02.063440.log
/content/.config/logs/2018.11.20/18.06.15.887476.log
/content/.config/logs/2018.11.20/18.06.14.858997.log
/content/.config/logs/2018.11.25
/content/.config/logs/2018.11.25/14.32.52.436227.log
/content/.config/logs/2018.11.25/14.33.39.325097.log
/content/.config/.last_update_check.json
/content/.config/configurations
/content/.config/configurations/config_default
/content/.config/gce
/content/.config/config_sentinel
/content/.config/active_config
/content/.config/access_tokens.db
/content/.config/credentials.db
/content/.config/legacy_credentials
/content/.config/legacy_credentials/fkarinapassos@gmail.com
/content/.config/legacy_credentials/fkarinapassos@gmail.com/.boto
/content/.config/legacy_credentials/fkarinapassos@gmail.com/adc.json
/content/samp

In [0]:
task3_img = 'ISIC2018_Task3_Training_Input'
task3_validation_img = 'ISIC2018_Task3_Validation_Input'
task3_test_img = 'ISIC2018_Task3_Test_Input'

task3_gt = 'ISIC2018_Task3_Training_GroundTruth'

In [0]:
task3_img_dir = os.path.join(data_dir, task3_img)
task3_validation_img_dir = os.path.join(data_dir, task3_validation_img)
task3_test_img_dir = os.path.join(data_dir, task3_test_img)

task3_gt_dir = os.path.join(data_dir, task3_gt)

In [16]:
##Moving and Unzipping the Images:

import shutil

shutil.move('ISIC2018_Task3_Test_Input.zip', '/content/datasets/ISIC2018/data/')
shutil.move('ISIC2018_Task3_Training_Input.zip', '/content/datasets/ISIC2018/data/')
shutil.move('ISIC2018_Task3_Validation_Input.zip', '/content/datasets/ISIC2018/data/')
shutil.move('ISIC2018_Task3_Training_GroundTruth.zip', '/content/datasets/ISIC2018/data/')

'/content/datasets/ISIC2018/data/ISIC2018_Task3_Training_GroundTruth.zip'

In [17]:

!cd '/content/datasets/ISIC2018/data/' && unzip -q 'ISIC2018_Task3_Training_Input.zip'
!cd '/content/datasets/ISIC2018/data/' && unzip -q 'ISIC2018_Task3_Test_Input.zip'
!cd '/content/datasets/ISIC2018/data/' && unzip -q 'ISIC2018_Task3_Validation_Input.zip'
!cd '/content/datasets/ISIC2018/data/' && unzip -q 'ISIC2018_Task3_Training_GroundTruth.zip'

!ls -hl '/content/datasets/ISIC2018/data/'

total 3.1G
drwxrwxr-x 2 root root  60K Jun  9 18:44 ISIC2018_Task3_Test_Input
-rw-r--r-- 1 root root 402M Nov 25 14:34 ISIC2018_Task3_Test_Input.zip
drwxrwxr-x 2 root root 4.0K Apr  2  2018 ISIC2018_Task3_Training_GroundTruth
-rw-r--r-- 1 root root  36K Nov 25 14:34 ISIC2018_Task3_Training_GroundTruth.zip
drwxrwxr-x 2 root root 336K Apr  2  2018 ISIC2018_Task3_Training_Input
-rw-r--r-- 1 root root 2.6G Nov 25 14:34 ISIC2018_Task3_Training_Input.zip
drwxrwxr-x 2 root root  12K Jun  9 18:29 ISIC2018_Task3_Validation_Input
-rw-r--r-- 1 root root  51M Nov 25 14:34 ISIC2018_Task3_Validation_Input.zip


In [18]:
import shutil
shutil.move('ISIC2018_Task3_Training_LesionGroupings.csv', '/content/datasets/ISIC2018/data/ISIC2018_Task3_Training_GroundTruth/')

'/content/datasets/ISIC2018/data/ISIC2018_Task3_Training_GroundTruth/ISIC2018_Task3_Training_LesionGroupings.csv'

In [19]:
!find '/content'

/content
/content/.config
/content/.config/logs
/content/.config/logs/2018.11.20
/content/.config/logs/2018.11.20/18.05.57.262344.log
/content/.config/logs/2018.11.20/18.06.10.838966.log
/content/.config/logs/2018.11.20/18.06.02.063440.log
/content/.config/logs/2018.11.20/18.06.15.887476.log
/content/.config/logs/2018.11.20/18.06.14.858997.log
/content/.config/logs/2018.11.25
/content/.config/logs/2018.11.25/14.32.52.436227.log
/content/.config/logs/2018.11.25/14.33.39.325097.log
/content/.config/.last_update_check.json
/content/.config/configurations
/content/.config/configurations/config_default
/content/.config/gce
/content/.config/config_sentinel
/content/.config/active_config
/content/.config/access_tokens.db
/content/.config/credentials.db
/content/.config/legacy_credentials
/content/.config/legacy_credentials/fkarinapassos@gmail.com
/content/.config/legacy_credentials/fkarinapassos@gmail.com/.boto
/content/.config/legacy_credentials/fkarinapassos@gmail.com/adc.json
/content/samp

In [0]:
import os
import numpy as np
from tqdm import tqdm
from skimage import io
from skimage import transform
import pandas as pd
import shutil, glob

In [0]:
task3_image_ids = list()
if os.path.isdir(task3_img_dir):
    task3_image_ids = [fname.rsplit('.', maxsplit=1)[0] for fname in os.listdir(task3_img_dir)
                       if fname.startswith('ISIC') and fname.lower().endswith('.jpg')]
    task3_image_ids.sort()
    

task3_validation_image_ids = list()
if os.path.isdir(task3_validation_img_dir):
    task3_validation_image_ids = [fname.rsplit('.', maxsplit=1)[0] for fname in os.listdir(task3_validation_img_dir)
                                  if fname.startswith('ISIC') and fname.lower().endswith('.jpg')]
    task3_validation_image_ids.sort()
    

task3_test_image_ids = list()
if os.path.isdir(task3_test_img_dir):
    task3_test_image_ids = [fname.rsplit('.', maxsplit=1)[0] for fname in os.listdir(task3_test_img_dir)
                                  if fname.startswith('ISIC') and fname.lower().endswith('.jpg')]
    task3_test_image_ids.sort()


In [22]:
#verificando que se llenen las listas con los id

task3_image_ids

['ISIC_0024306',
 'ISIC_0024307',
 'ISIC_0024308',
 'ISIC_0024309',
 'ISIC_0024310',
 'ISIC_0024311',
 'ISIC_0024312',
 'ISIC_0024313',
 'ISIC_0024314',
 'ISIC_0024315',
 'ISIC_0024316',
 'ISIC_0024317',
 'ISIC_0024318',
 'ISIC_0024319',
 'ISIC_0024320',
 'ISIC_0024321',
 'ISIC_0024322',
 'ISIC_0024323',
 'ISIC_0024324',
 'ISIC_0024325',
 'ISIC_0024326',
 'ISIC_0024327',
 'ISIC_0024328',
 'ISIC_0024329',
 'ISIC_0024330',
 'ISIC_0024331',
 'ISIC_0024332',
 'ISIC_0024333',
 'ISIC_0024334',
 'ISIC_0024335',
 'ISIC_0024336',
 'ISIC_0024337',
 'ISIC_0024338',
 'ISIC_0024339',
 'ISIC_0024340',
 'ISIC_0024341',
 'ISIC_0024342',
 'ISIC_0024343',
 'ISIC_0024344',
 'ISIC_0024345',
 'ISIC_0024346',
 'ISIC_0024347',
 'ISIC_0024348',
 'ISIC_0024349',
 'ISIC_0024350',
 'ISIC_0024351',
 'ISIC_0024352',
 'ISIC_0024353',
 'ISIC_0024354',
 'ISIC_0024355',
 'ISIC_0024356',
 'ISIC_0024357',
 'ISIC_0024358',
 'ISIC_0024359',
 'ISIC_0024360',
 'ISIC_0024361',
 'ISIC_0024362',
 'ISIC_0024363',
 'ISIC_0024364

In [0]:
task3_gt_fname = 'ISIC2018_Task3_Training_GroundTruth.csv'
task3_sup_fname = 'ISIC2018_Task3_Training_LesionGroupings.csv'

In [0]:
task3_images_npy_prefix = 'task3_images'
task3_validation_images_npy_prefix = 'task3_validation_images'
task3_test_images_npy_prefix = 'task3_test_images'

task3_gt_npy_prefix = 'task3_labels'

In [0]:
MEL = 0  # Melanoma
NV = 1  # Melanocytic nevus
BCC = 2  # Basal cell carcinoma
AKIEC = 3  # Actinic keratosis / Bowen's disease (intraepithelial carcinoma)
BKL = 4  # Benign keratosis (solar lentigo / seborrheic keratosis / lichen planus-like keratosis)
DF = 5  # Dermatofibroma
VASC = 6  # Vascular lesion


classes = [MEL, NV, BCC, AKIEC, BKL, DF, VASC]
class_names = ['MEL', 'NV', 'BCC', 'AKIEC', 'BKL', 'DF', 'VASC']

**Resizing all the images to 224x224x3 and allocate them into a numpy file:**

In [0]:
def load_image_by_id(image_id, fname_fn, from_dir, output_size=None, return_size=False):
    img_fnames = fname_fn(image_id)
    if isinstance(img_fnames, str):
        img_fnames = [img_fnames, ]

    assert isinstance(img_fnames, tuple) or isinstance(img_fnames, list)

    images = []
    image_sizes = []

    for img_fname in img_fnames:
        img_fname = os.path.join(from_dir, img_fname)
        if not os.path.exists(img_fname):
            raise FileNotFoundError('img %s not found' % img_fname)
        image = io.imread(img_fname)

        image_sizes.append(np.asarray(image.shape[:2]))

        if output_size:
            image = transform.resize(image, (output_size, output_size),
                                     order=1, mode='constant',
                                     cval=0, clip=True,
                                     preserve_range=True) 
                                     
        image = image.astype(np.uint8)
        images.append(image)

    if return_size:
        if len(images) == 1:
            return images[0], image_sizes[0]
        else:
            return np.stack(images, axis=-1), image_sizes

    if len(images) == 1:
        return images[0]
    else:
        return np.stack(images, axis=-1) # masks
      

In [0]:
def load_images(image_ids, from_dir, output_size=None, fname_fn=None, verbose=True, return_size=False):
    images = []

    if verbose:
        print('loading images from', from_dir)

    if return_size:

        image_sizes = []
        for image_id in tqdm(image_ids):
            image, image_size = load_image_by_id(image_id,
                                                 from_dir=from_dir,
                                                 output_size=output_size,
                                                 fname_fn=fname_fn,
                                                 return_size=True)
            images.append(image)
            image_sizes.append(image_size)

        return images, image_sizes


    else:
        for image_id in tqdm(image_ids):
            image = load_image_by_id(image_id,
                                     from_dir=from_dir,
                                     output_size=output_size,
                                     fname_fn=fname_fn)
            images.append(image)

        return images

In [0]:
def load_task3_training_images(output_size=None):
    suffix = '' if output_size is None else '_%d' % output_size
    images_npy_filename = os.path.join(small_data_dir, '%s%s.npy' % (task3_images_npy_prefix, suffix))

    if os.path.exists(images_npy_filename):
        images = np.load(images_npy_filename)
    else:
        images = load_images(image_ids=task3_image_ids,
                             from_dir=task3_img_dir,
                             output_size=output_size,
                             fname_fn=lambda x: '%s.jpg' % x)
        images = np.stack(images).astype(np.uint8)
        np.save(images_npy_filename, images)
    return images

In [0]:
def load_task3_validation_images(output_size=None):
    suffix = '' if output_size is None else '_%d' % output_size
    images_npy_filename = os.path.join(small_data_dir, '%s%s.npy' % (task3_validation_images_npy_prefix, suffix))

    if os.path.exists(images_npy_filename):
        images = np.load(images_npy_filename)
    else:
        images = load_images(image_ids=task3_validation_image_ids,
                             from_dir=task3_validation_img_dir,
                             output_size=output_size,
                             fname_fn=lambda x: '%s.jpg' % x)
        images = np.stack(images).astype(np.uint8)
        np.save(images_npy_filename, images)
    return images

In [0]:
def load_task3_test_images(output_size=None):

    suffix = '' if output_size is None else '_%d' % output_size
    images_npy_filename = os.path.join(small_data_dir, '%s%s.npy' % (task3_test_images_npy_prefix, suffix))

    if os.path.exists(images_npy_filename):

        images = np.load(images_npy_filename)

    else:

        images = load_images(image_ids=task3_test_image_ids,
                             from_dir=task3_test_img_dir,
                             output_size=output_size,
                             fname_fn=lambda x: '%s.jpg' % x)
        images = np.stack(images).astype(np.uint8)
        np.save(images_npy_filename, images)

    return images

In [0]:
def load_task3_training_labels():
    # image, MEL, NV, BCC, AKIEC, BKL, DF, VASC
    labels = []
    with open(os.path.join(task3_gt_dir, task3_gt_fname), 'r') as f:
        for i, line in tqdm(enumerate(f.readlines()[1:])):
            fields = line.strip().split(',')
            labels.append([eval(field) for field in fields[1:]])
        labels = np.stack(labels, axis=0)
    return labels

In [0]:
def partition_task3_data(x, y, k=5, i=0, test_split=1. / 6, seed=42):
    assert isinstance(k, int) and isinstance(i, int) and 0 <= i < k

    fname = os.path.join(task3_gt_dir, task3_sup_fname)
    assert os.path.exists(fname)

    df = pd.read_csv(os.path.join(task3_gt_dir, task3_sup_fname))
    grouped = df.groupby('lesion_id', sort=True)
    lesion_ids = []
    for name, group in grouped:
        image_ids = group.image.tolist()
        lesion_ids.append([name, image_ids])

    # shuffle lesion ids
    np.random.seed(seed)
    n = len(lesion_ids)
    indices = np.random.permutation(n)

    image_ids = [image_id for idx in indices for image_id in lesion_ids[idx][1]]
    n = len(image_ids)
    n_set = int(n * (1. - test_split)) // k
    # divide the data into (k + 1) sets, -1 is test set, [0, k) are for train and validation
    indices = [i for i in range(k) for _ in range(n_set)] + [-1] * (n - n_set * k)

    indices = list(zip(indices, image_ids))
    indices.sort(key=lambda x: x[1])
    indices = np.array([idx for idx, image_id in indices], dtype=np.uint)

    valid_indices = (indices == i)
    test_indices = (indices == -1)
    train_indices = ~(valid_indices | test_indices)

    x_valid = x[valid_indices]
    y_valid = y[valid_indices]

    x_train = x[train_indices]
    y_train = y[train_indices]

    x_test = x[test_indices]
    y_test = y[test_indices]

    return (x_train, y_train), (x_valid, y_valid), (x_test, y_test)


In [0]:
def load_training_data(task_idx,
                       output_size=None,
                       num_partitions=5,
                       idx_partition=0,
                       test_split=0.):
    assert isinstance(task_idx, int) and 0 < task_idx <= 3
    if task_idx == 3:
        x = load_task3_training_images(output_size=output_size)
        y = load_task3_training_labels()
        return partition_task3_data(x=x, y=y, k=num_partitions, i=idx_partition, test_split=test_split)

    else:
        print ("task_idx invalid")

In [0]:
def load_validation_data(task_idx, output_size=None):
    assert isinstance(task_idx, int) and 0 < task_idx <= 3
    if task_idx == 3:
        return load_task3_validation_images(output_size=output_size), task3_validation_image_ids
    else:
        print ("task_idx invalid")

In [0]:
def load_test_data(task_idx, output_size=None):
    assert isinstance(task_idx, int) and 0 < task_idx <= 3
    if task_idx == 3:
        return load_task3_test_images(output_size=output_size), task3_test_image_ids
    else:
        print ("task_idx invalid")

In [0]:
def partition_data(x, y, k=5, i=0, test_split=1. / 6, seed=42):
    assert isinstance(k, int) and isinstance(i, int) and 0 <= i < k

    n = x.shape[0]

    n_set = int(n * (1. - test_split)) // k
    # divide the data into (k + 1) sets, -1 is test set, [0, k) are for train and validation
    indices = np.array([i for i in range(k) for _ in range(n_set)] +
                       [-1] * (n - n_set * k),
                       dtype=np.int8)

    np.random.seed(seed)
    np.random.shuffle(indices)

    valid_indices = (indices == i)
    test_indices = (indices == -1)
    train_indices = ~(valid_indices | test_indices)

    x_valid = x[valid_indices]
    y_valid = y[valid_indices]

    x_train = x[train_indices]
    y_train = y[train_indices]

    x_test = x[test_indices]
    y_test = y[test_indices]

    return (x_train, y_train), (x_valid, y_valid), (x_test, y_test)

In [37]:
#Este primer paso de cargar la data suele tardar cerca de 15min:
if __name__ == '__main__':

     _, _, _ = load_training_data(task_idx=3, output_size=224)
  
     images, image_names = load_validation_data(task_idx=3, output_size=224)
     images, image_names = load_test_data(task_idx=3, output_size=224)

  0%|          | 1/10015 [00:00<25:27,  6.55it/s]

loading images from /content/datasets/ISIC2018/data/ISIC2018_Task3_Training_Input


100%|██████████| 10015/10015 [10:15<00:00, 16.28it/s]
10015it [00:00, 25937.72it/s]
  2%|▏         | 4/193 [00:00<00:05, 36.56it/s]

loading images from /content/datasets/ISIC2018/data/ISIC2018_Task3_Validation_Input


  0%|          | 4/1512 [00:00<00:39, 38.26it/s]

loading images from /content/datasets/ISIC2018/data/ISIC2018_Task3_Test_Input


100%|██████████| 1512/1512 [00:40<00:00, 36.73it/s]


## Data visualization

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Button
from matplotlib import colors

In [0]:
def plot_mask(axis_in, img_in, mask_in, title_in):
    mask_colors = ['k', 'r', 'b', 'g', 'y', 'c', 'm']
    img = img_in.copy()
    axis_in.clear()
    if mask_in.shape[2] > 1:
        mask_max = np.argmax(mask_in, axis=2)
        for mask_idx in range(1, mask_in.shape[2]):
            img[mask_max == mask_idx, :] = np.round(
                np.asarray(colors.colorConverter.to_rgb(mask_colors[mask_idx])) * 255)
    else:
        img[mask_in[:, :, 0] > 0.5, :] = np.round(np.asarray(colors.colorConverter.to_rgb('y')) * 255)

    axis_in.imshow(img)

    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    font_weight = 'bold'
    font_color = 'k'

    fontdict = {'family': 'serif',
                'color': font_color,
                'weight': font_weight,
                'size': 14,
                }

    # place a text box in upper left in axes coords
    axis_in.text(0.35, 0.95, title_in, transform=axis_in.transAxes, fontdict=fontdict, verticalalignment='top',
                 bbox=props)

In [0]:
class BatchVisualization(object):
    def __init__(self, images,
                 true_masks=None, pred_masks=None,
                 true_labels=None, pred_labels=None,
                 legends=None, **fig_kwargs):

        self.images = images
        self.n_images = self.images.shape[0]
        self.true_masks = true_masks
        self.pred_masks = pred_masks
        self.true_labels = true_labels
        self.pred_labels = pred_labels
        self.legends = legends
        if true_labels is not None:
            self.ncols = fig_kwargs.get('ncols', 3)
            self.nrows = fig_kwargs.get('nrows', 3)
        else:  # Mask predictions
            self.ncols = fig_kwargs.get('ncols', 1)
            self.nrows = fig_kwargs.get('nrows', 3)

        self.batch_size = self.ncols * self.nrows
        self.start_idx = 0
        self.num_plots_per_image = 1

        if self.true_masks is not None:
            self.num_plots_per_image += 1

        if self.pred_masks is not None:
            self.num_plots_per_image += 1

        if self.true_masks is not None and len(self.true_masks.shape) == 3:
            self.true_masks = np.expand_dims(self.true_masks, axis=3)

        if self.pred_masks is not None and len(self.pred_masks.shape) == 3:
            self.pred_masks = np.expand_dims(self.pred_masks, axis=3)

        self.mask_type = fig_kwargs.get('mask_type', 'contour')

    def __call__(self, *args, **kwargs):

        self.fig, self.ax = plt.subplots(figsize=(10, 8),
                                         nrows=self.nrows,
                                         ncols=self.num_plots_per_image * self.ncols,
                                         sharex='all',
                                         sharey='all',
                                         gridspec_kw={'hspace': 0.,
                                                      'wspace': 0.
                                                      }
                                         )

        self.ax = np.ravel(self.ax)

        for ax in self.ax:
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            ax.set_aspect('equal')
            ax.axis('off')

        self.fig.subplots_adjust(bottom=0.2, hspace=0, wspace=0)

        self.axprev = plt.axes([0.7, 0.05, 0.1, 0.075])
        self.axnext = plt.axes([0.81, 0.05, 0.1, 0.075])

        self.bnext = Button(self.axnext, 'Next')
        self.bnext.on_clicked(self.next)
        self.bprev = Button(self.axprev, 'Previous')
        self.bprev.on_clicked(self.prev)

        self.start_idx = 0

        self.update_batch()
        self.update_buttons()
        plt.show(block=True)

    def next(self, event):
        self.start_idx += self.batch_size
        self.update_batch()
        self.update_buttons()
        plt.show(block=True)

    def prev(self, event):
        self.start_idx -= self.batch_size
        self.update_batch()
        self.update_buttons()
        plt.show(block=True)

    def update_buttons(self):
        if self.start_idx + self.batch_size < self.n_images:
            self.axnext.set_visible(True)
            self.bnext.set_active(True)
        else:
            self.axnext.set_visible(False)
            self.bnext.set_active(False)

        if self.start_idx - self.batch_size >= 0:
            self.bprev.set_active(True)
            self.axprev.set_visible(True)
        else:
            self.bprev.set_active(False)
            self.axprev.set_visible(False)

    def update_batch(self):

        for ax_idx, image_idx in enumerate(range(self.start_idx,
                                                 min(self.start_idx + self.batch_size,
                                                     self.n_images))):

            img_ax_idx = ax_idx * self.num_plots_per_image

            self.ax[img_ax_idx].clear()
            self.ax[img_ax_idx].imshow(self.images[image_idx])

            if self.true_masks is not None:
                img_ax_idx += 1

                plot_mask(axis_in=self.ax[img_ax_idx],
                          img_in=self.images[image_idx],
                          mask_in=self.true_masks[image_idx],
                          title_in='GT mask')

            if self.true_labels is not None or self.pred_labels is not None:

                true_label = None if self.true_labels is None else class_names[np.argmax(self.true_labels[image_idx])]
                pred_label = None if self.pred_labels is None else class_names[np.argmax(self.pred_labels[image_idx])]

                if true_label is not None and pred_label is not None:
                    label = '%s -> %s' % (true_label, pred_label)
                    font_color = 'darkgreen' if true_label == pred_label else 'darkred'
                else:
                    label = '%s' % true_label if pred_label is None else pred_label
                    font_color = 'k'

                # these are matplotlib.patch.Patch properties
                props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
                font_weight = 'bold'

                fontdict = {'family': 'serif',
                            'color': font_color,
                            'weight': font_weight,
                            'size': 14,
                            }

                # place a text box in upper left in axes coords
                ax = self.ax[img_ax_idx]
                ax.text(0.35, 0.95, label, transform=ax.transAxes, fontdict=fontdict, verticalalignment='top',
                        bbox=props)

            if self.pred_masks is not None:
                img_ax_idx += 1

                plot_mask(axis_in=self.ax[img_ax_idx],
                          img_in=self.images[image_idx],
                          mask_in=self.pred_masks[image_idx],
                          title_in='Pred mask')

        if self.legends:
            plt.figlegend()

        for ax in self.ax:
            ax.set_xticklabels([])
            ax.set_yticklabels([])
            ax.set_aspect('equal')
            ax.axis('off')

        plt.draw()

## Functions and Parameters 

In [0]:
import numpy as np
import sys
import logging
import sys
import os
import inspect

from sklearn.metrics import confusion_matrix, precision_recall_fscore_support
from sklearn.utils import compute_class_weight as sk_compute_class_weight


In [0]:
### misc_print_utils:

class PrintColors:

    GREEN = "\033[0;32m"
    BLUE = "\033[1;34m"
    RED = "\033[1;31m"

    HEADER = '\033[95m'
    OK_BLUE = '\033[94m'
    OK_GREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    END_COLOR = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'


def log_variable(var_name, var_value):
    print('{: <20} : {}'.format(var_name, var_value))


def print_confusion_matrix(cm, labels):
    """print for confusion matrixes"""

    columnwidth = max([len(x) for x in labels] + [12])
    # Print header
    print()
    first_cell = "True\Pred"
    print("|%{0}s|".format(columnwidth - 2) % first_cell, end="")
    for label in labels:
        print("%{0}s|".format(columnwidth -1) % label, end="")
    print()

    first_cell = "-------"
    print("|%{0}s|".format(columnwidth-2) % first_cell, end="")
    for _ in labels:
        print("%{0}s|".format(columnwidth-1) % first_cell, end="")
    print()

    # Print rows
    for i, label1 in enumerate(labels):
        print("|%{0}s|".format(columnwidth - 2) % label1, end="")
        for j in range(len(labels)):
            cell = "%{0}.2f|".format(columnwidth-1) % cm[i, j]
            if i == len(labels) - 1 or j == len(labels) - 1:
                cell = "%{0}d|".format(columnwidth-1) % cm[i, j]
                if i == j:
                    print("%{0}s|".format(columnwidth-1) % ' ', end="")
                else:
                    print(PrintColors.BLUE + cell + PrintColors.END_COLOR, end="")
            elif i == j:
                print(PrintColors.GREEN + cell + PrintColors.END_COLOR, end="")
            else:
                print(PrintColors.RED + cell + PrintColors.END_COLOR, end="")

        print()


def print_precision_recall(precision, recall, labels):
    columnwidth = max([len(x) for x in labels] + [12])
    # Print header
    print()
    first_cell = " "
    print("|%{0}s|".format(columnwidth-2) % first_cell, end="")
    for label in labels:
        print("%{0}s|".format(columnwidth-1) % label, end="")
    print("%{0}s|".format(columnwidth-1) % 'MEAN', end="")
    print()

    first_cell = "-------"
    print("|%{0}s|".format(columnwidth-2) % first_cell, end="")
    for _ in labels:
        print("%{0}s|".format(columnwidth-1) % first_cell, end="")
    print("%{0}s|".format(columnwidth-1) % first_cell, end="")
    print()

    # print precision
    print("|%{0}s|".format(columnwidth-2) % 'precision', end="")
    for j in range(len(labels)):
        cell = "%{0}.3f|".format(columnwidth-1) % precision[j]
        print(PrintColors.GREEN + cell + PrintColors.END_COLOR, end="")

    cell = "%{0}.3f|".format(columnwidth-1) % np.mean(precision)
    print(PrintColors.BLUE + cell + PrintColors.END_COLOR, end="")

    print()

    # print recall
    print("|%{0}s|".format(columnwidth-2) % 'recall', end="")
    for j in range(len(labels)):
        cell = "%{0}.3f|".format(columnwidth-1) % recall[j]
        print(PrintColors.GREEN + cell + PrintColors.END_COLOR, end="")

    cell = "%{0}.3f|".format(columnwidth-1) % np.mean(recall)
    print(PrintColors.BLUE + cell + PrintColors.END_COLOR, end="")

    print('')


class Tee(object):
    def __init__(self, stream1, stream2):
        self.stream1 = stream1
        self.stream2 = stream2
        self.__missing_method_name = None # Hack!

    def __getattribute__(self, name):
        return object.__getattribute__(self, name)

    def __getattr__(self, name):
        self.__missing_method_name = name # Could also be a property
        return getattr(self, '__methodmissing__')

    def __methodmissing__(self, *args, **kwargs):
        # Emit method call to the log copy
        callable2 = getattr(self.stream2, self.__missing_method_name)
        callable2(*args, **kwargs)

        # Emit method call to stdout (stream 1)
        callable1 = getattr(self.stream1, self.__missing_method_name)
        return callable1(*args, **kwargs)

In [0]:
### misc_eval_utils:


def get_confusion_matrix(y_true, y_pred, norm_cm=True, print_cm=True):
    true_class = np.argmax(y_true, axis=1)
    pred_class = np.argmax(y_pred, axis=1)

    cnf_mat = confusion_matrix(true_class, pred_class, labels=classes)

    total_cnf_mat = np.zeros(shape=(cnf_mat.shape[0] + 1, cnf_mat.shape[1] + 1), dtype=np.float)
    total_cnf_mat[0:cnf_mat.shape[0], 0:cnf_mat.shape[1]] = cnf_mat

    for i_row in range(cnf_mat.shape[0]):
        total_cnf_mat[i_row, -1] = np.sum(total_cnf_mat[i_row, 0:-1])

    for i_col in range(cnf_mat.shape[1]):
        total_cnf_mat[-1, i_col] = np.sum(total_cnf_mat[0:-1, i_col])

    if norm_cm:
        cnf_mat = cnf_mat/(cnf_mat.astype(np.float).sum(axis=1)[:, np.newaxis] + 0.001)

    total_cnf_mat[0:cnf_mat.shape[0], 0:cnf_mat.shape[1]] = cnf_mat

    if print_cm:
        print_confusion_matrix(cm=total_cnf_mat, labels=class_names + ['TOTAL', ])

    return cnf_mat


def get_precision_recall(y_true, y_pred, print_pr=True):

    true_class = np.argmax(y_true, axis=1)
    pred_class = np.argmax(y_pred, axis=1)
    precision, recall, _, _ = precision_recall_fscore_support(y_true=true_class,
                                                              y_pred=pred_class,
                                                              labels=classes,
                                                              warn_for=())
    if print_pr:
        print_precision_recall(precision=precision, recall=recall, labels=class_names)

    return precision, recall


def compute_class_weights(y, wt_type='balanced', return_dict=True):
    # need to check if y is one hot
    if len(y.shape) > 1:
        y = y.argmax(axis=-1)

    assert wt_type in ['ones', 'balanced', 'balanced-sqrt'], 'Weight type not supported'

    classes = np.unique(y)
    class_weights = np.ones(shape=classes.shape[0])

    if wt_type == 'balanced' or wt_type == 'balanced-sqrt':

        class_weights = sk_compute_class_weight(class_weight='balanced',
                                                classes=classes,
                                                y=y)
        if wt_type == 'balanced-sqrt':
            class_weights = np.sqrt(class_weights)

    if return_dict:
        class_weights = dict([(i, w) for i, w in enumerate(class_weights)])

    return class_weights


def jaccard(y_true, y_pred):
    intersect = np.sum(y_true * y_pred) # Intersection points
    union = np.sum(y_true) + np.sum(y_pred)  # Union points
    return (float(intersect))/(union - intersect +  1e-7)


def compute_jaccard(y_true, y_pred):

    mean_jaccard = 0.
    thresholded_jaccard = 0.

    for im_index in range(y_pred.shape[0]):

        current_jaccard = jaccard(y_true=y_true[im_index], y_pred=y_pred[im_index])

        mean_jaccard += current_jaccard
        thresholded_jaccard += 0 if current_jaccard < 0.65 else current_jaccard

    mean_jaccard = mean_jaccard/y_pred.shape[0]
    thresholded_jaccard = thresholded_jaccard/y_pred.shape[0]

    return mean_jaccard, thresholded_jaccard

In [0]:
### misc_filename_utils:

import os
import errno


def get_run_dir(run_name):
    dirname = os.path.join(model_data_dir, run_name)
    try:
        os.makedirs(dirname)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise

    return dirname


def get_weights_filename(run_name):
    dirname = get_run_dir(run_name)
    weights_filename = os.path.join(dirname, '%s.hdf5' % run_name)
    return weights_filename


def get_csv_filename(run_name):
    dirname = get_run_dir(run_name)
    csv_filename = os.path.join(dirname, '%s.csv' % run_name)
    return csv_filename


def get_log_filename(run_name):
    dirname = get_run_dir(run_name)
    log_filename = os.path.join(dirname, '%s_log.txt' % run_name)
    return log_filename


def get_model_summary_filename(run_name):
    dirname = get_run_dir(run_name)
    filename = os.path.join(dirname, '%s.txt' % run_name)
    return filename


def get_model_image_filename(run_name):
    dirname = get_run_dir(run_name)
    filename = os.path.join(dirname, '%s.png' % run_name)
    return filename


def get_json_filename(run_name):
    dirname = get_run_dir(run_name)
    json_filename = os.path.join(dirname, '%s.json' % run_name)
    return json_filename


def get_model_config_filename(run_name):
    dirname = get_run_dir(run_name)
    config_filename = os.path.join(dirname, '%s.pkl' % run_name)
    return config_filename


In [46]:
### callback:

from matplotlib import pyplot as plt
from keras.callbacks import Callback, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from keras.utils.vis_utils import plot_model

plt.ion()


class PlotModel(Callback):
    def __init__(self, filename):
        super(PlotModel, self).__init__()
        self.filename = filename

    def on_train_begin(self, logs=None):
        plot_model(self.model,
                   to_file=self.filename,
                   show_shapes=True,
                   show_layer_names=True)


class ModelSummary(Callback):
    def __init__(self, filename):
        self.filename = filename
        super(ModelSummary, self).__init__()

    def on_train_begin(self, logs=None):
        with open(self.filename, 'w') as file:
            self.model.summary(print_fn=lambda x: file.write(x + '\n'))


class ValidationPrediction(Callback):
    def __init__(self, show_confusion_matrix=False, **kwargs):
        super(ValidationPrediction, self).__init__()

        self.show_confusion_matrix = show_confusion_matrix

        self.visualize = kwargs.get('visualize', False)
        self.nrows = kwargs.get('nrows', 5)
        self.ncols = kwargs.get('ncols', 5)
        self.mask_colors = kwargs.get('mask_colors', ['r', 'b', 'g', 'c', 'm', 'y'])
        self.n_choices = self.nrows * self.ncols

        # for display purposes
        self.fig = None
        self.ax = None
        self.indices = None

        self.confusion_fig = None
        self.confusion_ax = None

        # setup
        self.y_true = None
        self.y_pred = None

    def on_epoch_end(self, epoch, logs=None):
        self.make_predictions()
        if self.show_confusion_matrix:
            self.view_confusion_matrix()

        if self.visualize:
            self.visualize_validation_prediction()

    def make_predictions(self):
        self.y_pred = self.model.predict(self.validation_data[0])
        self.y_true = self.validation_data[1]

    def view_confusion_matrix(self):
        _ = get_confusion_matrix(y_true=self.y_true, y_pred=self.y_pred, print_cm=True)
        get_precision_recall(y_true=self.y_true, y_pred=self.y_pred)

    def visualize_validation_prediction(self):
        if self.fig is None:
            self.fig, self.ax = plt.subplots(figsize=(5, 5),
                                             nrows=self.nrows,
                                             ncols=self.ncols,
                                             sharex='all',
                                             sharey='all')

            n_samples = self.validation_data[0].shape[0]

            self.indices = np.random.choice(np.arange(n_samples),
                                            size=self.n_choices,
                                            replace=False)

            x = self.validation_data[0][[self.indices]]

            for i, ax in enumerate(self.ax.flatten()):
                ax.clear()
                ax.imshow(x[i])

            plt.show()

        y_true = self.y_true[self.indices]
        y_pred = self.y_pred[self.indices]

        # check to see if masks, or labels
        try:
            n_imgs, img_height, img_width, img_channel = y_true.shape
            masks = np.concatenate(y_pred, y_true)
            labels = None
        except ValueError:
            n_imgs, n_classes = y_true.shape
            labels = (y_pred, y_true)
            masks = None

        for i, ax in enumerate(self.ax.flatten()):

            if masks is not None:
                if len(masks.shape) == 2:
                    masks = np.expand_dims(masks, axis=2)

                for j in range(masks.shape[2]):
                    mask = masks[:, :, j]
                    if mask.max() > 0:
                        ax.contour(mask, [127.5, ],
                                   colors=self.mask_colors[j])

            if labels is not None:
                y_pred_i = labels[0][i].argmax()
                y_true_i = labels[1][i].argmax()
                ax.set_title('%s/%s' % (y_pred_i, y_true_i))
                if y_pred_i != y_true_i:
                    color = 'red' if y_true_i == 0 else 'magenta'
                else:
                    color = 'green'

                for axis in ['top', 'bottom', 'left', 'right']:
                    ax.spines[axis].set_linewidth(2.0)
                    ax.spines[axis].set_color(color)

            ax.set_xticklabels([])
            ax.set_yticklabels([])
            ax.set_aspect('equal')


        self.fig.canvas.draw()
        self.fig.canvas.flush_events()
        plt.pause(3)


def config_cls_callbacks(run_name=None):
    callbacks = [
        ValidationPrediction(show_confusion_matrix=True),
        ReduceLROnPlateau(monitor='val_loss',
                          factor=0.25,
                          patience=2,
                          verbose=1,
                          mode='auto',
                          min_lr=1e-7)
    ]
    if run_name:
        callbacks.extend([
            ModelCheckpoint(get_weights_filename(run_name),
                            monitor='val_loss',
                            save_best_only=True,
                            save_weights_only=True,
                            verbose=True),
            CSVLogger(filename=get_csv_filename(run_name))
        ])
    return callbacks


def config_seg_callbacks(run_name=None):
    callbacks = [
        ValidationPrediction(show_confusion_matrix=False),
        ReduceLROnPlateau(monitor='val_loss',
                          factor=0.5,
                          patience=2,
                          verbose=1,
                          mode='auto',
                          min_lr=1e-7),
    ]
    if run_name:
        callbacks.extend([
            ModelCheckpoint(get_weights_filename(run_name),
                            monitor='val_loss',
                            save_best_only=True,
                            save_weights_only=True,
                            verbose=True),
            CSVLogger(filename=get_csv_filename(run_name))
        ])
    return callbacks

Using TensorFlow backend.


In [0]:
### metrics:

from keras import backend as K

def pixelwise_precision(num_classes=1):
    def binary_pixelwise_precision(y_true, y_pred):
        true_pos = K.sum(K.abs(y_true * y_pred), axis=[1, 2, 3])
        total_pos = K.sum(K.abs(y_pred), axis=[1, 2, 3])
        return true_pos / K.clip(total_pos, K.epsilon(), None)

    def categorical_pixelwise_precision(y_true, y_pred):
        true_pos = K.sum(K.abs(y_true * y_pred), axis=[1, 2])
        total_pos = K.sum(K.abs(y_pred), axis=[1, 2])
        return true_pos / K.clip(total_pos, K.epsilon(), None)

    if num_classes == 1:
        return binary_pixelwise_precision
    else:
        return categorical_pixelwise_precision


def pixelwise_recall(num_classes=1):
    return pixelwise_sensitivity(num_classes)


def pixelwise_sensitivity(num_classes=1):
    def binary_pixelwise_sensitivity(y_true, y_pred):
        """
        true positive rate, probability of detection
        sensitivity = Nro of true positives / (Nro of true positives + Nro of false negatives)
        :param y_true:
        :param y_pred:
        :return:
        """

        y_true = K.round(y_true)
        true_pos = K.sum(K.abs(y_true * y_pred), axis=[1, 2, 3])
        total_pos = K.sum(K.abs(y_true), axis=[1, 2, 3])
        return true_pos / K.clip(total_pos, K.epsilon(), None)

    def categorical_pixelwise_sensitivity(y_true, y_pred):
        true_pos = K.sum(K.abs(y_true * y_pred), axis=[1, 2])
        total_pos = K.sum(K.abs(y_true), axis=[1, 2])
        return K.mean(true_pos / K.clip(total_pos, K.epsilon(), None), axis=-1)

    if num_classes == 1:
        return binary_pixelwise_sensitivity
    else:
        return categorical_pixelwise_sensitivity


def pixelwise_specificity(num_classes=1):
    """
    true negative rate
    the proportion of negatives that are correctly identified as such
    specificity = Nro of true negatives / (Nro of true negatives + Nro of false positives)
    :param y_true:  ground truth
    :param y_pred: prediction
    :return:
    """

    def binary_pixelwise_specificity(y_true, y_pred):
        true_neg = K.sum(K.abs((1. - y_true) * (1. - y_pred)), axis=[1, 2, 3])
        total_neg = K.sum(K.abs(1. - y_true), axis=[1, 2, 3])
        return true_neg / K.clip(total_neg, K.epsilon(), None)

    def categorical_pixelwise_specificity(y_true, y_pred):
        y_true, y_pred = y_true[..., 1:], y_pred[..., 1:]
        true_neg = K.sum(K.abs((1. - y_true) * (1. - y_pred)), axis=[1, 2])
        total_neg = K.sum(K.abs(1. - y_true), axis=[1, 2])
        return true_neg / K.clip(total_neg, K.epsilon(), None)
    if num_classes == 1:
        return binary_pixelwise_specificity
    else:
        return categorical_pixelwise_specificity


def dice_coeff(num_classes=1):
    def binary_dice_coeff(y_true, y_pred):
        """
                DSC = (2 * |X & Y|)/ (|X|+ |Y|)
                    = 2 * sum(|A*B|)/(sum(|A|)+sum(|B|))
        :param y_true: ground truth
        :param y_pred: prediction
        :return:
        """

        intersection = K.sum(K.abs(y_true * y_pred), axis=[1, 2, 3])
        union = K.sum(K.abs(y_true) + K.abs(y_pred), axis=[1, 2, 3])
        dice = 2 * intersection / K.clip(union, K.epsilon(), None)
        return dice

    def categorical_dice_coeff(y_true, y_pred):

        intersection = K.sum(K.abs(y_true * y_pred), axis=[1, 2])
        union = K.sum(K.abs(y_true) + K.abs(y_pred), axis=[1, 2])
        dice = 2 * intersection / K.clip(union, K.epsilon(), None)
        return K.mean(dice, axis=-1)

    if num_classes == 1:
        return binary_dice_coeff
    else:
        return categorical_dice_coeff


def class_jaccard_index(idx):
    def jaccard_index(y_true, y_pred):
        y_true, y_pred = y_true[..., idx], y_pred[..., idx]
        y_true = K.round(y_true)
        y_pred = K.round(y_pred)
        # Adding all three axis to average across images before dividing
        # See https://forum.isic-archive.com/t/task-2-evaluation-and-superpixel-generation/417/2
        intersection = K.sum(K.abs(y_true * y_pred), axis=[0, 1, 2])
        sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=[0, 1, 2])
        jac = intersection / K.clip(sum_ - intersection, K.epsilon(), None)
        return jac
    return jaccard_index


def jaccard_index(num_classes):
    """
    Jaccard index for semantic segmentation, also known as the intersection-over-union.
        This loss is useful when you have unbalanced numbers of pixels within an image
        because it gives all classes equal weight. However, it is not the defacto
        standard for image segmentation.
        For example, assume you are trying to predict if each pixel is cat, dog, or background.
        You have 80% background pixels, 10% dog, and 10% cat. If the model predicts 100% background
        should it be be 80% right (as with categorical cross entropy) or 30% (with this loss)?
        The loss has been modified to have a smooth gradient as it converges on zero.
        This has been shifted so it converges on 0 and is smoothed to avoid exploding
        or disappearing gradient.
        Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
                = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))
        """

    def binary_jaccard_index(y_true, y_pred):
        y_true = K.round(y_true)
        y_pred = K.round(y_pred)
        intersection = K.sum(K.abs(y_true * y_pred), axis=[1, 2, 3])
        union = K.sum(K.abs(y_true) + K.abs(y_pred), axis=[1, 2, 3])
        iou = intersection / K.clip(union - intersection, K.epsilon(), None)
        return iou

    def categorical_jaccard_index(y_true, y_pred):
        y_true = K.round(y_true)
        y_pred = K.round(y_pred)
        intersection = K.abs(y_true * y_pred)
        union = K.abs(y_true) + K.abs(y_pred)

        intersection = K.sum(intersection, axis=[0, 1, 2])
        union = K.sum(union, axis=[0, 1, 2])

        iou = intersection / K.clip(union - intersection, K.epsilon(), None)
        return iou

    if num_classes == 1:
        return binary_jaccard_index
    else:
        return categorical_jaccard_index


In [0]:
### submodels_segmentation:

import keras

# from models.layers import SubPixelUpscaling
from keras import backend as K
from keras.layers import LeakyReLU
from keras.layers import Conv2D
from keras.layers import Cropping2D
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import UpSampling2D
from keras.layers import Conv2DTranspose
from keras.utils import conv_utils


def __conv_block(nb_filters,
                 activation='relu',
                 block_prefix=None):

    options = {
        'kernel_size': 3,
        'strides': 1,
        'padding': 'same',
    }

    nb_layers_per_block = 1 if isinstance(nb_filters, int) else len(nb_filters)
    nb_filters = conv_utils.normalize_tuple(nb_filters, nb_layers_per_block, 'nb_filters')

    def block(x):
        for i, n in enumerate(nb_filters):
            x = Conv2D(filters=nb_filters[i],
                       name=name_or_none(block_prefix, '_conv%d' % (i+1)),
                       **options)(x)

            if activation.lower() == 'leakyrelu':
                x = LeakyReLU(alpha=0.33)(x)
            else:
                x = Activation(activation)(x)
        return x
    return block


def __transition_up_block(nb_filters,
                          merge_size,
                          upsampling_type='deconv',
                          block_prefix=None):
    """Adds an upsampling block. Upsampling operation relies on the the type parameter.
    # Arguments
        ip: input keras tensor
        nb_filters: integer, the dimensionality of the output space
            (i.e. the number output of filters in the convolution)
        type: can be 'upsample', 'subpixel', 'deconv'. Determines
            type of upsampling performed
        block_prefix: str, for block unique naming
    # Input shape
        4D tensor with shape:
        `(samples, channels, rows, cols)` if data_format='channels_first'
        or 4D tensor with shape:
        `(samples, rows, cols, channels)` if data_format='channels_last'.
    # Output shape
        4D tensor with shape:
        `(samples, nb_filter, rows * 2, cols * 2)` if data_format='channels_first'
        or 4D tensor with shape:
        `(samples, rows * 2, cols * 2, nb_filter)` if data_format='channels_last'.
    # Returns
        a keras tensor
    """
    options = {
        'padding': 'same'
    }

    if upsampling_type not in {'upsample', 'subpixel', 'deconv'}:
        raise ValueError('upsampling_type must be in  {`upsample`, `subpixel`, `deconv`}: %s' % str(upsampling_type))

    merge_size = conv_utils.normalize_tuple(merge_size, 2, 'merge_size')

    def block(ip):
        try:
            src, dst = ip
        except TypeError:
            src = ip
            dst = None

        # copy and crop
        if K.image_data_format() == 'channels_last':
            indices = slice(1, 3)
            channel_axis = -1
        else:
            indices = slice(2, 4)
            channel_axis = 1

        src_height, src_width = K.get_variable_shape(src)[indices]

        target_height, target_width = merge_size
        scale_factor = ((target_height + src_height - 1) // src_height,
                        (target_width + src_width - 1) // src_width)

        # upsample and crop
        if upsampling_type == 'upsample':
            x = UpSampling2D(size=scale_factor,
                             name=name_or_none(block_prefix, '_upsampling'))(src)
            x = Conv2D(nb_filters, (2, 2),
                       activation='relu', padding='same', name=name_or_none(block_prefix, '_conv'))(x)
        elif upsampling_type == 'subpixel':
            x = Conv2D(nb_filters, (2, 2),
                       activation='relu', padding='same', name=name_or_none(block_prefix, '_conv'))(src)
            x = SubPixelUpscaling(scale_factor=scale_factor,
                                  name=name_or_none(block_prefix, '_subpixel'))(x)
        else:
            x = Conv2DTranspose(nb_filters, (2, 2), strides=scale_factor,
                                name=name_or_none(block_prefix, '_deconv'),
                                **options)(src)

        if src_height * scale_factor[0] > target_height or src_width * scale_factor[1] > target_width:
            height_padding, width_padding = (src_height - target_height) // 2, (src_width - target_width) // 2
            x = Cropping2D(cropping=(height_padding, width_padding),
                           name=name_or_none(block_prefix, 'crop1'))(x)

        if dst is None:
            return x

        dst_height, dst_width = K.get_variable_shape(dst)[indices]

        # copy and crop
        if dst_height > target_height or dst_width > target_width:
            height_padding, width_padding = ((dst_height - target_height) // 2, (dst_width - target_width) // 2)
            dst = Cropping2D(cropping=(height_padding, width_padding),
                             name=name_or_none(block_prefix, 'crop2'))(dst)

        x = Concatenate(axis=channel_axis, name=name_or_none(block_prefix, '_merge'))([x, dst])

        return x

    return block


# todo: can be made more efficient
def __normalize_target_size(curr_size, target_size, scale_factor):
    while curr_size < target_size:
        target_size //= scale_factor
    return target_size


def default_decoder_model(features,
                          num_classes=1,
                          output_size=224,
                          scale_factor=2,
                          init_nb_filters=64,
                          growth_rate=2,
                          nb_layers_per_block=2,
                          max_nb_filters=512,
                          upsampling_type='deconv',
                          activation='relu',
                          bottleneck=False,
                          use_activation=True,
                          include_top=True):
    """
    :param features:            list of features from encoder
    :param output_size:         size of the output segmentation mask
    :param num_classes:         The number of classes of pixels.
    :param init_nb_filters:     Number of filters for last conv block.
    :param growth_rate:         The rate at which the number of filters grow from block to block
    :param nb_layers_per_block: Number of layers for each conv block.
    :param max_nb_filters:      max # of filters
    :param scale_factor:        The rate at which the size grows
    :param upsampling_type:     Upsampling type
    :param activation:          activation of conv blocks
    :param use_activation:      whether to use activation of output layer
    :param include_top:         whether to use the top layer
    :param bottleneck:          add bottleneck at the output of encoder
    :return: A keras.model.Model that predicts classes
    """

    output_size = conv_utils.normalize_tuple(output_size, 2, 'output_size')
    output_height, output_width = output_size

    __init_nb_filters = init_nb_filters
    indices = slice(1, 3) if K.image_data_format() == 'channels_last' else slice(2, 4)
    channel = 3 if K.image_data_format() == 'channels_last' else 1

    nb_features = len(features)
    feature_shapes = [K.get_variable_shape(feature) for feature in features]
    feature_sizes = [feature_shape[indices] for feature_shape in feature_shapes]

    feature_height, feature_width = feature_sizes[0]
    if feature_height < output_height or feature_width < output_width:
        __init_nb_filters = int(__init_nb_filters * growth_rate)

    if bottleneck:
        for i in range(nb_features - 1, -1, -1):
            feature_shape = feature_shapes[i]
            nb_filters = int(__init_nb_filters * (growth_rate ** i))
            nb_filters = min(nb_filters, max_nb_filters)
            if feature_shape[channel] > nb_filters:
                features[i] = Conv2D(nb_filters, 1,
                                     padding='same',
                                     activation='relu',
                                     name='feature%d_bottleneck' % (i+1))(features[i])

    nb_layers_per_block = conv_utils.normalize_tuple(nb_layers_per_block, nb_features, 'nb_layers_per_block')

    x = features[-1]

    for i in range(nb_features-1, 0, -1):
        dst = features[i-1]
        dst_height, dst_width = feature_sizes[i-1]

        merge_size = __normalize_target_size(dst_height, output_height, scale_factor)
        if dst_width != dst_height:
            merge_size = (merge_size, __normalize_target_size(dst_width, output_width, scale_factor))

        nb_filters = int(__init_nb_filters * (growth_rate ** (i-1)))
        nb_filters = min(nb_filters, max_nb_filters)

        x = __transition_up_block(nb_filters=nb_filters,
                                  merge_size=merge_size,
                                  block_prefix='feature%d' % (i+1),
                                  upsampling_type=upsampling_type)([x, dst])

        x = __conv_block(nb_filters=conv_utils.normalize_tuple(nb_filters,
                                                               nb_layers_per_block[i-1],
                                                               'nb_filters'),
                         activation=activation,
                         block_prefix='feature%d' % i)(x)

    if __init_nb_filters > init_nb_filters:
        x = __transition_up_block(nb_filters=init_nb_filters,
                                  merge_size=output_size,
                                  block_prefix='decoder_block%d' % (nb_features+1),
                                  upsampling_type=upsampling_type)(x)

        x = __conv_block(nb_filters=[init_nb_filters],
                         activation=activation,
                         block_prefix='feature%d' %(nb_features + 1))(x)

    if include_top:
        x = Conv2D(num_classes, (1, 1), activation='linear', name='predictions')(x)
        if use_activation:
            output_activation = 'sigmoid' if num_classes == 1 else 'softmax'
            x = Activation(output_activation, name='outputs')(x)

    return x

In [0]:
### Initializers:

##calculo de probabilidad:

import keras
import numpy as np
import math


class PriorProbability(keras.initializers.Initializer):
    """Initializer applies a prior probability"""

    def __init__(self, probability=0.01):
        self.probability = probability

    def get_config(self):
        return {
            'probability': self.probability
        }

    def __call__(self, shape, dtype=None):
        # set bias to -log((1 - p)/p) for foreground
        result = np.ones(shape, dtype=dtype) * -math.log((1 - self.probability) / self.probability)
        return result

In [0]:
### submodels_classification:

import keras
from keras import Input, backend as K, regularizers


def default_classification_model(input_shape=None,
                                 input_tensor=None,
                                 num_classes=7,
                                 num_dense_layers=2,
                                 num_dense_units=256,
                                 dropout_rate=0.,
                                 pooling=None,
                                 use_output_activation=True,
                                 kernel_regularizer=None):

    """
    :param kernel_regularizer: l1 or l2 or none regularization
    :param num_classes:             # of classes to predict a score for each feature level.
    :param input_shape:             Input shape
    :param input_tensor:            Input tensor
    :param num_dense_layers:         Number of dense layers before the output layer
    :param num_dense_units:              The number of filters to use in the layers in the classification submodel.
    :param dropout_rate:            Dropout Rate
    :param pooling:                 which pooling to use at conv output
    :param use_output_activation:   whether to use output activation
    :return: A keras.model.Model that predicts class
    """

    if input_tensor is None:
        img_input = Input(shape=input_shape)
    else:
        if not K.is_keras_tensor(input_tensor):
            img_input = Input(tensor=input_tensor, shape=input_shape)
        else:
            img_input = input_tensor

    assert kernel_regularizer in [None, 'L1', 'L2', 'L1-L2'], \
        'Unknown regularizer %s' % kernel_regularizer

    if kernel_regularizer == 'L1':
        kernel_regularizer = regularizers.l1(1e-4)
    elif kernel_regularizer == 'L2':
        kernel_regularizer = regularizers.l2(1e-3)
    elif kernel_regularizer == 'L1-L2':
        kernel_regularizer = regularizers.l1_l2(l1=1e-4, l2=1e-3)

    assert pooling in {None, 'avg', 'max', 'flatten'}, 'Unknown pooling option %s' % pooling

    if pooling == 'avg':
        outputs = keras.layers.GlobalAveragePooling2D(name='avg_pool_our')(img_input)
    elif pooling == 'max':
        outputs = keras.layers.GlobalMaxPooling2D(name='max_pool_our')(img_input)
    else:
        outputs = keras.layers.Flatten(name='flatten_our')(img_input)

    if dropout_rate > 0.:
        outputs = keras.layers.Dropout(rate=dropout_rate)(outputs)

    for i in range(num_dense_layers):
        outputs = keras.layers.Dense(num_dense_units,
                                     activation='relu',
                                     name='fc%d' % (i + 1),
                                     kernel_regularizer=kernel_regularizer
                                     )(outputs)

        outputs = keras.layers.Dropout(rate=dropout_rate)(outputs)

    outputs = keras.layers.Dense(num_classes,
                                 name='predictions',
                                 kernel_regularizer=kernel_regularizer
                                 )(outputs)

    if use_output_activation:
        activation = 'sigmoid' if num_classes == 1 else 'softmax'
        outputs = keras.layers.Activation(activation, name='outputs')(outputs)

    return outputs

In [0]:
### paths:
# save the predicctions of validation set

import os
import inspect


def mkdir_if_not_exist(dir_list):
    for directory in dir_list:
        if not os.path.exists(directory):
            os.makedirs(directory)


curr_filename = inspect.getfile(inspect.currentframe())
root_dir = os.path.dirname(os.path.abspath(curr_filename))
model_data_dir = os.path.join(root_dir, 'model_data')
submission_dir = os.path.join(root_dir, 'submissions')

dir_to_make = [model_data_dir, submission_dir]
mkdir_if_not_exist(dir_list=dir_to_make)
mkdir_if_not_exist(dir_list=dir_to_make)

In [0]:
### misc_models_utils:

import os

import keras
from keras.models import load_model, model_from_json


def load_model_weights_from(model, weights, skip_mismatch):
    if weights is None:
        return

    if os.path.exists(weights):
        model.load_weights(weights, by_name=True, skip_mismatch=skip_mismatch)
        return

    weights_path = get_weights_filename(weights)
    if os.path.exists(weights_path):
        model.load_weights(weights_path, by_name=True, skip_mismatch=skip_mismatch)
        return

    raise ValueError('Unknown weights to load from!', weights, weights_path)


def save_model_to_run(model, run_name):
    json_path = get_json_filename(run_name)
    h5_path = get_weights_filename(run_name)

    with open(json_path, 'w') as json_file:
        json_file.write(model.to_json())

    model.save_weights(h5_path)


def load_model_from_run(backbone_name,
                        load_model_from,
                        load_weights_from=None,
                        skip_mismatch=True):
    b = backbone(backbone_name)
    json_path = get_json_filename(load_model_from)

    if not os.path.exists(json_path):
        h5_path = get_weights_filename(load_model_from)
        if not os.path.exists(h5_path):
            raise ValueError("run with name %s doesn't exist" % load_model_from)

        model = load_model(h5_path, custom_objects=b.custom_objects, compile=False)
    else:
        with open(json_path, 'r') as json_file:
            json_string = json_file.read()
        model = model_from_json(json_string, custom_objects=b.custom_objects)

        if load_weights_from:
            h5_path = get_weights_filename(load_weights_from)
            if os.path.exists(h5_path):
                model.load_weights(h5_path, by_name=True, skip_mismatch=skip_mismatch)
    return model


def freeze_model(model, layers=None):
    if layers is None:
        model.trainable = False
    else:
        for layer in layers:
            if isinstance(layer, int):
                layer = model.get_layer(index=layer)
            elif isinstance(layer, str):
                layer = model.get_layer(name=layer)
            else:
                raise ValueError('layer must be either an index or a string')
            layer.trainable = False

    return model



def plot_model(save_to_dir, model, name):
    filename = os.path.join(model_data_dir, save_to_dir, '%s.png' % name)
    keras.utils.plot_model(model,
                           to_file=filename,
                           show_shapes=True,
                           show_layer_names=True)


def name_or_none(prefix, name):
    return prefix + name if (prefix is not None and name is not None) else None


In [0]:
#### MODELS_BACKBONE:

class Backbone(object):
    """
    This class stores additional information on backbones
    """

    def __init__(self, backbone_name, **kwargs):
        """
        :param backbone_name: name of the backbone
        :param kwargs: user provided kwargs in case a custom base model is needed
        """
        
        self.custom_objects = {
            # included custom metrics in case the saved model need to be compiled
            'dice_coeff': dice_coeff,
            'jaccard_index': jaccard_index,
            'pixelwise_precision': pixelwise_precision,
            'pixelwise_specificity': pixelwise_specificity,
            'pixelwise_sensitivity': pixelwise_sensitivity,
            'PriorProbability': PriorProbability,
        }

        self.backbone_name = backbone_name
        self.backbone_options = kwargs
        self.scale_factor = 2
        self.validate()

    def build_base_model(self, inputs, **kwarg):
        raise NotImplementedError('backbone method not implemented')

    def classification_model(self,
                             input_shape=None,
                             input_padding=None,
                             submodel=None,
                             num_classes=7,
                             num_dense_layers=2,
                             num_dense_units=256,
                             dropout_rate=0.,
                             pooling=None,
                             use_output_activation=True,
                             kernel_regularizer=None,
                             use_activation=True,
                             include_top=True,
                             name='default_classification_model',
                             print_model_summary=False,
                             plot_model_summary=False,
                             load_from=None,
                             load_model_from=None,
                             load_weights_from=None,
                             save_to=None,
                             lr=1e-5,
                             ):
        """ Returns a classifier model using the correct backbone.
        """
        from keras import backend as K

        if load_from:
            model = load_model_from_run(self.backbone_name, load_from, load_from)
        elif load_model_from:
            model = load_model_from_run(self.backbone_name, load_model_from, load_weights_from)
        else:
            if K.image_data_format() == 'channels_last':
                input_shape = (224, 224, 3) if input_shape is None else input_shape
            else:
                input_shape = (3, 224, 224) if input_shape is None else input_shape

            inputs = keras.layers.Input(shape=input_shape)

            x = inputs
            if input_padding is not None:
                x = keras.layers.ZeroPadding2D(padding=input_padding)(x)

            base_model = self.build_base_model(inputs=x, **self.backbone_options)
            x = base_model.output

            if submodel is None:
                outputs = default_classification_model(input_tensor=x,
                                                       input_shape=base_model.output_shape[1:],
                                                       num_classes=num_classes,
                                                       num_dense_layers=num_dense_layers,
                                                       num_dense_units=num_dense_units,
                                                       dropout_rate=dropout_rate,
                                                       pooling=pooling,
                                                       use_output_activation=use_activation,
                                                       kernel_regularizer=kernel_regularizer)
            else:
                outputs = submodel(x)

            model = keras.models.Model(inputs=inputs, outputs=outputs, name=name)

            if load_weights_from:
                load_model_weights_from(model, load_weights_from, skip_mismatch=True)

        if print_model_summary:
            model.summary()

        if plot_model_summary: 
            plot_model(save_to_dir=save_to, model=model, name=name)

        if save_to:
            save_model_to_run(model, save_to)

        compile_model(model=model,
                      num_classes=num_classes,
                      metrics='acc',
                      loss='ce',
                      lr=lr)

        return model

    def segmentation_model(self,
                           load_from=None,
                           load_model_from=None,
                           load_weights_from=None,
                           save_to=None,
                           lr=1e-5,
                           loss='ce',
                           metrics=None,
                           print_model_summary=False,
                           plot_model_summary=False,
                           input_shape=None,
                           input_padding=None,
                           backbone_layer_names=None,
                           submodel=None,
                           modifier=None,
                           num_classes=1,
                           init_nb_filters=64,
                           growth_rate=2,
                           nb_layers_per_block=2,
                           max_nb_filters=512,
                           bottleneck=False,
                           upsampling_type='deconv',
                           activation='relu',
                           use_activation=True,
                           include_top=True,
                           prior_probability=0.5,
                           name='default_segmentation_model'):
        """
        Returns a segmentation model using the correct backbone
        """
        import keras

        if load_from:
            print('loading from', load_from)
            model = load_model_from_run(self.backbone_name, load_from, load_from)
        elif load_model_from:
            model = load_model_from_run(self.backbone_name, load_model_from, load_weights_from)
        else:
            inputs = keras.layers.Input(shape=input_shape)
            if keras.backend.image_data_format() == 'channels_last':
                indices = slice(0, 2)
                input_shape = (224, 224, 3) if input_shape is None else input_shape
            else:
                indices = slice(1, 3)
                input_shape = (3, 224, 224) if input_shape is None else input_shape

            output_size = input_shape[indices]

            x = inputs
            if input_padding is not None:
                x = keras.layers.ZeroPadding2D(padding=input_padding)(x)

            base_model = self.build_base_model(inputs=x, **self.backbone_options)

            if backbone_layer_names:
                backbone_layers = [base_model.get_layer(name=layer_name) for layer_name in backbone_layer_names]
                backbone_features = [backbone_layer.output for backbone_layer in backbone_layers]
            else:
                outputs = base_model.output
                backbone_features = [outputs, ]

            if submodel is None:
                outputs = default_decoder_model(features=backbone_features,
                                                num_classes=num_classes,
                                                output_size=output_size,
                                                scale_factor=self.scale_factor,
                                                init_nb_filters=init_nb_filters,
                                                growth_rate=growth_rate,
                                                nb_layers_per_block=nb_layers_per_block,
                                                max_nb_filters=max_nb_filters,
                                                upsampling_type=upsampling_type,
                                                bottleneck=bottleneck,
                                                activation=activation,
                                                use_activation=use_activation,
                                                include_top=False)
            else:
                outputs = submodel(backbone_features)

            if include_top:
                outputs = keras.layers.Conv2D(num_classes, (1, 1), activation='linear', name='predictions')(outputs)
                if use_activation:
                    output_activation = 'sigmoid' if num_classes == 1 else 'softmax'
                    outputs = keras.layers.Activation(output_activation, name='outputs')(outputs)

            model = keras.models.Model(inputs=inputs, outputs=outputs, name=name)
            if load_weights_from:
                load_model_weights_from(model, load_weights_from, skip_mismatch=True)

        if print_model_summary:
            model.summary()

        if plot_model_summary and save_to: 
            plot_model(save_to_dir=save_to, model=model, name=name)

        if save_to:
            save_model_to_run(model, save_to)

        if modifier:
            model = modifier(model)

        if metrics is None:
            metrics = ['acc',
                       'jaccard_index',
                       'pixelwise_sensitivity',
                       'pixelwise_specificity']

        compile_model(model=model,
                      num_classes=num_classes,
                      metrics=metrics,
                      loss=loss,
                      lr=lr)

        return model

    def validate(self):
        """ Checks whether the backbone string is correct.
        """
        raise NotImplementedError('backbone method not implemented')


def backbone(backbone_name, **kwargs):
    """
    Returns a backbone object.
    """
    if 'resnet' in backbone_name:
        return ResNetBackbone(backbone_name, **kwargs)
    else:
        raise NotImplementedError('Backbone class for  \'{}\' not implemented.'.format(backbone_name))

    return VGGBackbone(backbone_name, **kwargs)

  

def compile_model(model, num_classes, metrics, loss, lr):
    from keras.losses import binary_crossentropy
    from keras.losses import categorical_crossentropy

    from keras.metrics import binary_accuracy
    from keras.metrics import categorical_accuracy

    from keras.optimizers import Adam

    if isinstance(loss, str):
        if loss in {'ce', 'crossentropy'}:
            if num_classes == 1:
                loss = binary_crossentropy
            else:
                loss = categorical_crossentropy
        elif loss in {'focal', 'focal_loss'}:
            loss = focal_loss(num_classes)
        else:
            raise ValueError('unknown loss %s' % loss)

    if isinstance(metrics, str):
        metrics = [metrics, ]

    for i, metric in enumerate(metrics):
        if not isinstance(metric, str):
            continue
        elif metric == 'acc':
            metrics[i] = binary_accuracy if num_classes == 1 else categorical_accuracy
        elif metric == 'jaccard_index':
            metrics[i] = jaccard_index(num_classes)
        elif metric == 'jaccard_index0':
            metrics[i] = class_jaccard_index(0)
        elif metric == 'jaccard_index1':
            metrics[i] = class_jaccard_index(1)
        elif metric == 'jaccard_index2':
            metrics[i] = class_jaccard_index(2)
        elif metric == 'jaccard_index3':
            metrics[i] = class_jaccard_index(3)
        elif metric == 'jaccard_index4':
            metrics[i] = class_jaccard_index(4)
        elif metric == 'jaccard_index5':
            metrics[i] = class_jaccard_index(5)
        elif metric == 'dice_coeff':
            metrics[i] = dice_coeff(num_classes)
        elif metric == 'pixelwise_precision':
            metrics[i] = pixelwise_precision(num_classes)
        elif metric == 'pixelwise_sensitivity':
            metrics[i] = pixelwise_sensitivity(num_classes)
        elif metric == 'pixelwise_specificity':
            metrics[i] = pixelwise_specificity(num_classes)
        elif metric == 'pixelwise_recall':
            metrics[i] = pixelwise_recall(num_classes)
        else:
            raise ValueError('metric %s not recognized' % metric)

    model.compile(optimizer=Adam(lr=lr),
                  loss=loss,
                  metrics=metrics)

In [0]:
### Resnet50 Model:


from keras.applications import resnet50 as keras_resnet50


class ResNetBackbone(Backbone):
    def __init__(self, backbone_name='resnet50'):
        super(ResNetBackbone, self).__init__(backbone_name)
        self.custom_objects['keras_resnet50'] = keras_resnet50

    def build_base_model(self, inputs, **kwarg):
        if self.backbone_name == 'resnet50':
            inputs = keras.layers.Lambda(lambda x: keras_resnet50.preprocess_input(x))(inputs)
            resnet = keras_resnet50.ResNet50(input_tensor=inputs,
                                             include_top=False,
                                             weights='imagenet')
        else:
            raise ValueError("Backbone '{}' not recognized.".format(self.backbone_name))

        return resnet

    def classification_model(self,
                             num_dense_layers=0,
                             num_dense_units=0,
                             dropout_rate=0.2,
                             pooling='avg',
                             name='default_resnet_classification_model',
                             **kwargs):
        """ Returns a classifier model using the correct backbone.
        """
        return super(ResNetBackbone, self).classification_model(num_dense_layers=num_dense_layers,
                                                                num_dense_units=num_dense_units,
                                                                dropout_rate=dropout_rate,
                                                                pooling=pooling,
                                                                name=name,
                                                                **kwargs)

    def validate(self):
        """ Checks whether the backbone string is correct.
        """
        allowed_backbones = ['resnet50', ]

        if self.backbone_name not in allowed_backbones:
            raise ValueError('Backbone (\'{}\') not in allowed backbones ({}).'.format(self.backbone_name, allowed_backbones))
            


class ResNetBackbone(Backbone):
    def __init__(self, backbone_name='resnet50'):
        super(ResNetBackbone, self).__init__(backbone_name)
        self.custom_objects['keras_resnet50'] = keras_resnet50

    def build_base_model(self, inputs, **kwarg):
        # create the vgg backbone
        if self.backbone_name == 'resnet50':
            inputs = keras.layers.Lambda(lambda x: keras_resnet50.preprocess_input(x))(inputs)
            resnet = keras_resnet50.ResNet50(input_tensor=inputs,
                                             include_top=False,
                                             weights='imagenet')
        else:
            raise ValueError("Backbone '{}' not recognized.".format(self.backbone_name))

        return resnet

    def classification_model(self,
                             num_dense_layers=0,
                             num_dense_units=0,
                             dropout_rate=0.2,
                             pooling='avg',
                             name='default_resnet_classification_model',
                             **kwargs):
        """ Returns a classifier model using the correct backbone.
        """
        return super(ResNetBackbone, self).classification_model(num_dense_layers=num_dense_layers,
                                                                num_dense_units=num_dense_units,
                                                                dropout_rate=dropout_rate,
                                                                pooling=pooling,
                                                                name=name,
                                                                **kwargs)

    def validate(self):
        """ Checks whether the backbone string is correct.
        """
        allowed_backbones = ['resnet50', ]

        if self.backbone_name not in allowed_backbones:
            raise ValueError('Backbone (\'{}\') not in allowed backbone ({}).'.format(self.backbone_name,
                                                                                       allowed_backbones))

In [0]:
### losses:

from keras import backend as K
import tensorflow as tf

def focal_loss(alpha=0.25, gamma=2.0, num_classes=1):
    def binary_focal_loss(y_true, y_pred):
        # compute the focal loss
        alpha_factor = K.ones_like(y_true) * alpha
        alpha_factor = tf.where(K.equal(y_true, 1), alpha_factor, 1 - alpha_factor)
        focal_weight = tf.where(K.equal(y_true, 1), 1 - y_pred, y_pred)
        focal_weight = alpha_factor * focal_weight ** gamma

        loss = focal_weight * K.binary_crossentropy(y_true, y_pred)
        return loss


    def categorical_focal_loss(y_true, y_pred):
        alpha_factor = K.ones_like(y_true) * alpha
        alpha_factor = tf.where(K.equal(y_true, 1), alpha_factor, 1. - alpha_factor)
        focal_weight = tf.where(K.equal(y_true, 1), 1. - y_pred, y_pred)
        focal_weight = alpha_factor * focal_weight ** gamma

        loss = focal_weight * K.categorical_crossentropy(y_true, y_pred)
        normalizer = K.sum(K.abs(y_true), axis=[1, 2])
        loss = K.sum(loss, axis=[1, 2])/K.maximum(1., normalizer)
        return K.mean(loss)

    if num_classes == 1:
        return binary_focal_loss
    else:
        return categorical_focal_loss


In [0]:
## misc_predicction_utils:

import numpy as np


def inv_sigmoid(x):
    x = np.clip(x, eps, 1 - eps)
    return np.log(x / (1. - x))


eps = np.finfo(np.float32).eps


def sigmoid(x):
    return 1. / (1. + np.exp(-x))


def rot90_4D(images):
    return np.transpose(np.flip(images, axis=2), axes=[0, 2, 1, 3])


def rot180_4D(images):
    return np.flip(np.flip(images, axis=1), axis=2)


def rot270_4D(images):
    return np.flip(np.transpose(images, axes=[0, 2, 1, 3]), axis=2)


def fliplr_4D(images):
    return np.flip(images, axis=2)


def flipud_4D(images):
    return np.flip(images, axis=1)


def cyclic_pooling(y,
                   y_rot90,
                   y_rot180,
                   y_rot270,
                   y_fliplr=None,
                   y_rot90_fliplr=None,
                   y_rot180_fliplr=None,
                   y_rot270_fliplr=None,
                   use_sigmoid=True,
                   data_type='img'):

    if len(y.shape) == 3:
        data_type = 'mask'
        y = y[..., None]
        y_rot90 = y_rot90[..., None]
        y_rot180 = y_rot180[..., None]
        y_rot270 = y_rot270[..., None]
        y_fliplr = y_fliplr[..., None]
        y_rot90_fliplr = y_rot90_fliplr[..., None]
        y_rot180_fliplr = y_rot180_fliplr[..., None]
        y_rot270_fliplr = y_rot270_fliplr[..., None]

    y1 = rot270_4D(y_rot90)
    y2 = rot180_4D(y_rot180)
    y3 = rot90_4D(y_rot270)

    y4 = fliplr_4D(y_fliplr)
    y5 = rot270_4D(fliplr_4D(y_rot90_fliplr))
    y6 = rot180_4D(fliplr_4D(y_rot180_fliplr))
    y7 = rot90_4D(fliplr_4D(y_rot270_fliplr))

    y_stacked = np.stack([y, y1, y2, y3, y4, y5, y6, y7], axis=0)

    if use_sigmoid:
        y_stacked = inv_sigmoid(y_stacked)

    y_stacked = np.mean(y_stacked, axis=0, keepdims=False)
    y_stacked = sigmoid(y_stacked)

    if data_type == 'mask':
        y_stacked = y_stacked[..., 0]

    return y_stacked


def cyclic_stacking(x):
    assert len(x.shape) == 4
    x_rot90 = rot90_4D(x)
    x_rot180 = rot180_4D(x)
    x_rot270 = rot270_4D(x)

    x_fliplr = fliplr_4D(x)
    x_rot90_fliplr = fliplr_4D(x_rot90)
    x_rot180_fliplr = fliplr_4D(x_rot180)
    x_rot270_fliplr = fliplr_4D(x_rot270)

    return x, x_rot90, x_rot180, x_rot270, x_fliplr, x_rot90_fliplr, x_rot180_fliplr, x_rot270_fliplr
  

## **Trainning the model**


*   The loss used for training was categorical cross-entropy loss along with L1 regularization loss to reduce over-fitting.

*   The Resnet50 model was trained using 5-fold cross validation. The model is based on this architecture:
![Resnet50 Model Architecture](https://www.researchgate.net/publication/324472161/figure/fig3/AS:614473802997760@1523513379024/Schematic-overview-of-basic-transfer-learning-algorithm-We-use-ResNet50-and-truncate-the.png)

*   We took into account the fact that multiple images of same lesion could be present in the training set, and avoided placing images with the same lesion ID in both training and validation sets. 
*   The Adam optimizer was used to train each fold for 10 epochs. Each epoch used a batch size of 32 with 500 steps. 

*   A learning rate of 10−4 was used.  Learning rate is halved when validation loss does not decrease for two epochs.  Further, early stopping is used to prevent overfitting the model. 
*   The images were augmented using these augmentations:

|      Augmentation|         Value|
|   -------|    -------|    -------|
| Horizontal flip|      p=0.5|      
|    Vertical flip|      p=0.5|     
|    Rotation|     -180º - 180º|   
|    Horizontal shift|    0 - 10%|   
|    Vertical shift|    0 - 10%|   
























In [0]:
## train classification model: (este paso suele tardar cerca de 1 hora) 

if __name__ == '__main__':

    from keras.preprocessing.image import ImageDataGenerator
    import sys

    backbone_name = 'resnet50'

    # Network architecture related params
    backbone_options = {}
    num_dense_layers = 1
    num_dense_units = 128
    pooling = 'avg'
    dropout_rate = 0.

    # Training related params
    dense_layer_regularizer = 'L1'
    class_wt_type = 'ones'
    lr = 1e-4

    num_folds = 5  

    for k_fold in range(num_folds):

        version = '0'

        run_name = 'task3_' + backbone_name + '_k' + str(k_fold) + '_v' + version
        # Set prev_run_name to continue training from a previous run
        prev_run_name = None

        logfile = open(get_log_filename(run_name=run_name), 'w+')
        original = sys.stdout
        sys.stdout = Tee(sys.stdout, logfile)

        (x_train, y_train), (x_valid, y_valid), _ = load_training_data(task_idx=3,
                                                                       output_size=224,
                                                                       num_partitions=num_folds,
                                                                       idx_partition=k_fold)

        debug_visualize = False

        if debug_visualize:
            x_train = x_train[:32]
            y_train = y_train[:32]

            x_valid = x_valid[:32]
            y_valid = y_valid[:32]

            bv = BatchVisualization(images=x_train, true_labels=y_train)
            bv()

        num_classes = y_train.shape[1]

        callbacks = config_cls_callbacks(run_name)

        model = backbone(backbone_name, **backbone_options).classification_model(
            input_shape=x_train.shape[1:],
            num_classes=num_classes,
            num_dense_layers=num_dense_layers,
            num_dense_units=num_dense_units,
            pooling=pooling,
            dropout_rate=dropout_rate,
            kernel_regularizer=dense_layer_regularizer,
            save_to=run_name,
            load_from=prev_run_name,
            print_model_summary=True,
            plot_model_summary=False,
            lr=lr)

        n_samples_train = x_train.shape[0]
        n_samples_valid = x_valid.shape[0]

        class_weights = compute_class_weights(y_train, wt_type=class_wt_type)

        batch_size = 32
        use_data_aug = True
        horizontal_flip = True
        vertical_flip = True
        rotation_angle = 180
        width_shift_range = 0.1
        height_shift_range = 0.1

        log_variable(var_name='num_dense_layers', var_value=num_dense_layers)
        log_variable(var_name='num_dense_units', var_value=num_dense_units)
        log_variable(var_name='dropout_rate', var_value=dropout_rate)
        log_variable(var_name='pooling', var_value=pooling)
        log_variable(var_name='class_wt_type', var_value=class_wt_type)
        log_variable(var_name='dense_layer_regularizer', var_value=dense_layer_regularizer)
        log_variable(var_name='class_wt_type', var_value=class_wt_type)
        log_variable(var_name='learning_rate', var_value=lr)
        log_variable(var_name='batch_size', var_value=batch_size)
        log_variable(var_name='use_data_aug', var_value=use_data_aug)

        if use_data_aug:

            log_variable(var_name='horizontal_flip', var_value=horizontal_flip)
            log_variable(var_name='vertical_flip', var_value=vertical_flip)
            log_variable(var_name='width_shift_range', var_value=width_shift_range)
            log_variable(var_name='height_shift_range', var_value=height_shift_range)
            log_variable(var_name='rotation_angle', var_value=rotation_angle)

        log_variable(var_name='n_samples_train', var_value=n_samples_train)
        log_variable(var_name='n_samples_valid', var_value=n_samples_valid)

        sys.stdout.flush()  # need to make sure everything gets written to file

        if use_data_aug:

            datagen = ImageDataGenerator(rotation_range=rotation_angle,
                                         horizontal_flip=horizontal_flip,
                                         vertical_flip=vertical_flip,
                                         width_shift_range=width_shift_range,
                                         height_shift_range=height_shift_range)

            model.fit_generator(generator=datagen.flow(x_train, y_train, batch_size=batch_size),
                                steps_per_epoch=x_train.shape[0] // batch_size * 2,
                                epochs=10,
                                initial_epoch=0,
                                verbose=1,
                                validation_data=(x_valid, y_valid),
                                callbacks=callbacks)
                                #, workers=8)
                                #, use_multiprocessing=True)

        else:

            model.fit(x=x_train,
                      y=y_train,
                      batch_size=batch_size,
                      epochs=30,
                      verbose=1,
                      validation_data=(x_valid, y_valid),
                      class_weight=class_weights,
                      shuffle=True,
                      callbacks=callbacks)

        sys.stdout = original

10015it [00:00, 29283.83it/s]
/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


94658560/94653016 [==============================] - 3s 0us/step
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 224, 224, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           lambda_1[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
____________________________________________

10015it [00:00, 27454.89it/s]


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 224, 224, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           lambda_2[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (

10015it [00:00, 26834.86it/s]


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 224, 224, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           lambda_3[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (

## Validation

NOTE: This step cannot be reached because the system automatically restarts (because memory fails) and clean up the variables obtained on the training phase.

In [0]:
## validation of classificacion model:     NOTE: This step cannot be reached because the system automatically restarts (because memory fails) 
                                                 #and clean up the variables obtained on the training phase.

if __name__ == '__main__':

    backbone_name = 'resnet50'
    k_fold = 0
    version = '0'
    run_name = 'task3_' + backbone_name + '_k' + str(k_fold) + '_v' + version

    _, (x, y_true), _ = load_training_data(task_idx=3,
                                           output_size=224,
                                           idx_partition=k_fold)

    model = backbone(backbone_name).classification_model(load_from=run_name)

    # max_num_images = 32
    max_num_images = x.shape[0]
    x = x[:max_num_images]
    y_true = y_true[:max_num_images]

    y_pred = model.predict(x)

    _ = get_confusion_matrix(y_true=y_true, y_pred=y_pred, print_cm=True)
    get_precision_recall(y_true=y_true, y_pred=y_pred)

    bv = BatchVisualization(images=x,
                            true_labels=y_true,
                            pred_labels=y_pred)
    bv()

## Predictions

To obtain classification predictions:  correct  prediction shown in green and wrong predictions shown in red. The predictions will be in submission csv file.

NOTE: This step cannot be reached because the system automatically restarts (because memory fails) and clean up the variables obtained on the training phase.

In [0]:
## predictions:   NOTE: This step cannot be reached because the system automatically restarts (because memory fails) 
                        #and clean up the variables obtained on the training phase.

import numpy as np


def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / np.sum(e_x, axis=1, keepdims=True)


if __name__ == '__main__':

    from keras import Model

    def task3_tta_predict(model, img_arr):
        img_arr_tta = cyclic_stacking(img_arr)
        pred_logits = np.zeros(shape=(img_arr.shape[0], 7))

        for _img_crops in img_arr_tta:
            pred_logits += model.predict(_img_crops)

        pred_logits = pred_logits/len(img_arr_tta)

        return pred_logits

    backbone_name = 'resnet50'
    version = '0'
    use_tta = False

    pred_set = 'validation'  # or test
    load_func = load_validation_data if pred_set == 'validation' else load_test_data
    images, image_names = load_func(task_idx=3, output_size=224)

    # max_num_images = 10
    max_num_images = images.shape[0]
    images = images[:max_num_images]
    image_names = image_names[:max_num_images]

    num_folds = 5

    print('Starting prediction for set %s with TTA set to %r with num_folds %d' % (pred_set, use_tta, num_folds))


    y_pred = np.zeros(shape=(max_num_images, 7))

    for k_fold in range(num_folds):
        print('Processing fold ', k_fold)
        run_name = 'task3_' + backbone_name + '_k' + str(k_fold) + '_v' + version
        model = backbone(backbone_name).classification_model(load_from=run_name)
        predictions_model = Model(inputs=model.input, outputs=model.get_layer('predictions').output)
        if use_tta:
            y_pred += task3_tta_predict(model=predictions_model, img_arr=images)
        else:
            y_pred += predictions_model.predict(images)

    y_pred = y_pred / num_folds
    y_prob = softmax(y_pred)

    print('Done predicting -- creating submission')

    submission_file = submission_dir + '/task3_' + pred_set + '_submission.csv'
    f = open(submission_file, 'w')
    f.write('image,MEL,NV,BCC,AKIEC,BKL,DF,VASC\n')

    for i_image, i_name in enumerate(image_names):
        i_line = i_name
        for i_cls in range(7):
            prob = y_prob[i_image, i_cls]
            if prob < 0.001:
                prob = 0.
            i_line += ',' + str(prob)

        i_line += '\n'
        f.write(i_line)  # Give your csv text here.

    f.close()

# Results and conclusions


*   According to the confusion matrix, the best performance on the training set was achieved on NV class (Melanocytic nevus) detection with accuracy close to 91%, the worst was obtained for class AKIEC (Actinic keratosis / intraepithelial carcinoma) with 30% accuracy of deteccion with 2% of samples incorrectly classified as BCC (Basal cell carcinoma) or DF (Dermatofibroma) classes. 

*   The confusion matrix and the precision/recall performance is shown on the validation set for one of the folds:

| True\Pred|        MEL|         NV|        BCC|      AKIEC|        BKL|         DF|       VASC|      TOTAL|
|   -------|    -------|    -------|    -------|    -------|    -------|    -------|    -------|    -------|
|       MEL|       0.64|       0.24|       0.01|       0.00|       0.10|       0.00|       0.01|        231|
|        NV|       0.06|       0.91|       0.01|       0.00|       0.03|       0.00|       0.00|       1324|
|       BCC|       0.01|       0.12|       0.70|       0.02|       0.12|       0.02|       0.00|         89|
|     AKIEC|       0.18|       0.01|       0.10|       0.30|       0.40|       0.00|       0.00|         67|
|       BKL|       0.12|       0.05|       0.04|       0.00|       0.78|       0.00|       0.00|        240|
|        DF|       0.11|       0.28|       0.06|       0.00|       0.17|       0.39|       0.00|         18|
|      VASC|       0.03|       0.06|       0.12|       0.00|       0.03|       0.00|       0.76|         34|
|     TOTAL|        271|       1285|         95|         23|        288|          9|         32|           |

|          |        MEL|         NV|        BCC|      AKIEC|        BKL|         DF|       VASC|       MEAN|
|   -------|    -------|    -------|    -------|    -------|    -------|    -------|    -------|    -------|
| precision|      0.546|      0.933|      0.653|      0.870|      0.653|      0.778|      0.812|      0.749|
|    recall|      0.641|      0.906|      0.697|      0.299|      0.783|      0.389|      0.765|      0.640|




*   Additionally it can be seen how after Epoch 5 the model is not improving and the learning rate is decreasing obtaining an average accuracy of 82% (perhaps the Loss is decreasing and the Accuracy is increasing epoch after epoch). 

*   Also, through the different epochs, we can appreciate that for MEL (Melanoma) class the achieved values for predicction were around 60% - 69%, and for almost cases the major error on the deteccion was to AKIEC and VASC (Vascular lesion) classes.

*  Comparing with Andrey Sorokin works, they applied a Mask R-CNN method combining Lesion boundary segmentation (Task 1) to build a lesion mask for every image in the training and validation set. The results obtained using these method were improved than results obtained using Resnet50 model without using mask.   

*   The model cannot be tested using validation data -193 images- without ground truth because the system was restarted automatically after the 3erd fold (10 epochs per fold) causing the loss of the variables. 

*  The use of a GPU Tesla 80 free resource in Google Colab, for these project was not enough to reach the fase to make predictions on validation and test set because the memory fails on system.
For projects with less demand of memory resources, Colab could be a powerful and practice tool.

## Further improvement

*  With more some time, try out other platform in order to avoid the memory fails and system restarts, and finally could make the prediccions of the test set.

*  Test others models for example InceptionV3 to training the net and compare the results.

*  Participate in the ISIC Challenge for next year.





   








